## MODULES

In [2]:
from shapely.validation import make_valid
import rasterio
from rasterio.features import shapes
import geopandas as gpd
import pandas as pd
import numpy as np
import os

## HYDROGEOLOGY

In [14]:
root_dir = "GEOCATMIN/commondata"
directories = [os.path.join(root_dir, d) for d in os.listdir(root_dir)
               if os.path.isdir(os.path.join(root_dir, d)) and not any(s in d for s in ["shp", "raster"])]

shp_paths = []
for directory in directories:
    shp_paths.extend([
        os.path.join(directory, f) for f in os.listdir(directory) if f.endswith(".shp")
    ])
    
Hydrogeologic_units = pd.concat([gpd.read_file(p) for p in shp_paths], ignore_index=True)

In [15]:
#print(shp_paths)
#print(Hydrogeologic_units)
#print(pd.unique(Hydrogeologic_units["NAME"]))
#Hydrogeologic_units["NAME"] = Hydrogeologic_units["NAME"].str.strip()
#print(pd.unique(Hydrogeologic_units["NAME"]))

In [16]:
replacements = {
    "Kis-a": "Kis-ar",
    "Js-l": "Js-la",
    "Jm-p": "Jm-pu",
    "Qp-b/c-anda": "Q-br2",
    "NQ-b-and": "N-br2",
    "NP-cbc-gnmg": "Js-la",
    "Jms-p": "Jms-pu",
    "Peo-h/t": "Peo-hu/t",
    "Js-g": "Js-gr",
    "Q-la": "Qpl-la",
    "Peo-h/qu": "Peo-hu/qu",
    "Peo-h/h": "Peo-hu/h",
    "N-and": "Js-la",
    "Nmp-ch-da": "N-da",
    "\tPeo-hu/h": "Peo-hu/h",
    "Nm-na/da": "N-da",
    "Nm-ta/da": "N-da",
    "Qp-b/am-tcri": "Q-br1",
    "NQ-b-tb": "Q-br1"
}
Hydrogeologic_units["NAME"] = Hydrogeologic_units["NAME"].replace(replacements)
Hydrogeologic_units.loc[Hydrogeologic_units["NAME_2"] == "PN-o", "NAME"] = "PN-o"

In [17]:
litologia = pd.read_excel("GEOCATMIN/resultados.xlsx", sheet_name=5)
nC = litologia.shape[1]

def calc_min(row):
    valores = row.iloc[2:nC]
    return valores.dropna().min() if not valores.dropna().empty else np.nan

def calc_max(row):
    valores = row.iloc[2:nC]
    return valores.dropna().max() if not valores.dropna().empty else np.nan

def calc_prom(row):
    unique_vals = row.iloc[2:4].dropna().tolist()
    paired_vals = row.iloc[4:nC]
    promedios = [np.nanmean(paired_vals[i:i+2]) for i in range(0, len(paired_vals), 2)]
    return np.nanmean(unique_vals + promedios)

litologia["Min"] = litologia.apply(calc_min, axis=1)
litologia["Max"] = litologia.apply(calc_max, axis=1)
litologia["Prom"] = litologia.apply(calc_prom, axis=1)
litologia = litologia[["Formación geológica", "Min", "Max", "Prom"]]
litologia.rename(columns={"Formación geológica": "NAME"}, inplace=True)

Hydrogeologic_units_join = Hydrogeologic_units.merge(litologia, on="NAME", how="left")
#print(list(Hydrogeologic_units_join.columns))
Hydrogeologic_units_join = Hydrogeologic_units_join[["NAME", "Min", "Max", "Prom", "geometry"]]
Hydrogeologic_units_join["geometry"] = Hydrogeologic_units_join["geometry"].apply(make_valid)

Hydrogeologic_units_join.to_file("1.PRODUCTOS/Hydrogeologic_units_join.shp")

['AREA', 'PERIMETER', 'GEOLUTMM_I', 'CODI', 'NAME', 'DESCRIP', 'HOJA', 'CODI_A', 'NAME_A', 'DESCRIP_A', 'UNIDAD', 'Shape_Leng', 'Shape_Area', 'area_1', 'NAME_2', 'CODI_1', 'Hidro_1', 'Hidro_2', 'Vulnerabil', 'geometry', 'Min', 'Max', 'Prom']


C:\Users\Karles\AppData\Local\Temp\ipykernel_3908\2454409418.py:15: RuntimeWarning: Mean of empty slice
  promedios = [np.nanmean(paired_vals[i:i+2]) for i in range(0, len(paired_vals), 2)]
C:\Users\Karles\AppData\Local\Temp\ipykernel_3908\2454409418.py:16: RuntimeWarning: Mean of empty slice
  return np.nanmean(unique_vals + promedios)


## MAPSWAT-SOIL

In [62]:
Soil_raster_MAPSWAT_path = "MAPSWAT/MapSWAT/SWAT_INPUT_MAPS/SOIL/SOIL.b1.tif"
Soil_MAPSWAT_properties = pd.read_csv("MAPSWAT/MapSWAT/SWAT_INPUT_MAPS/SOIL/DSOLMap_usersoil.csv")
#tmp2 = pd.read_csv("MAPSWAT/MapSWAT/SWAT_INPUT_MAPS/SOIL/DSOLMap_taxonomy.csv")
#tmp1.columns.values[0] = tmp2.columns.values[0] = "SOILID"
#tmp1.columns.values[0] = "SOILID"
#Soil_properties = tmp1.merge(tmp2, on = "SOILID", how = "left")
#print(list(Soil_properties.columns))

In [73]:
with rasterio.open(Soil_raster_MAPSWAT_path) as src:
    image = src.read(1)
    mask = image != src.nodata
    transform = src.transform

    results = (
        {"properties": {"OBJECTID": v}, "geometry": s}
        for s, v in shapes(image, mask=mask, transform=transform)
    )

Soil_shp_MAPSWAT = gpd.GeoDataFrame.from_features(results)
Soil_shp_MAPSWAT.crs = src.crs
Soil_shp_MAPSWAT = Soil_shp_MAPSWAT.dissolve(by="OBJECTID")
Soil_shp_MAPSWAT = Soil_shp_MAPSWAT.merge(Soil_MAPSWAT_properties, on = "OBJECTID", how = "left")

Soil_shp_MAPSWAT.to_file("1.PRODUCTOS/SOIL_MAPSWAT.shp")

## MERGING PRODUCTS

In [63]:
#SOIL_MAPSWAT = gpd.read_file("1.PRODUCTOS/SOIL_MAPSWAT.shp")
#Hydrogeologic_units_join = gpd.read_file("1.PRODUCTOS/Hydrogeologic_units_join.shp")
#Hydrogeologic_units_join_dis = Hydrogeologic_units_join.dissolve()

In [64]:
#MAPSWAT_Hydrogeologic_inte = gpd.overlay(SOIL_MAPSWAT, Hydrogeologic_units_join, how='intersection')

In [65]:
#MAPSWAT_Hydrogeologic_diff = gpd.overlay(SOIL_MAPSWAT, Hydrogeologic_units_join_dis, how='difference')

In [66]:
#SOIL_map = gpd.GeoDataFrame(pd.concat([MAPSWAT_Hydrogeologic_inte, MAPSWAT_Hydrogeologic_diff], ignore_index=True), crs=SOIL_MAPSWAT.crs)
#SOIL_map["OBJECTID"] = range(1, len(SOIL_map) + 1)
#SOIL_map["SNAM"] = SOIL_map["SNAM"] + '-' + SOIL_map["NAME"]
#SOIL_map.to_file("1.PRODUCTOS/SOIL_map.shp")

#### Equation 1
$Q_{soil} = - K(h)\times \frac{dh}{dz} - K(h)$

For saturated conditions

$\frac{dh}{dz} = 0$

Then, we can assume that K values from Hydrogeologic must be equal to the sum of K values form MAPSWAT by a constant "m"

$K_{hydrogeologic} = r\times\sum{ K_{MAPSWAT-layer}}$

#### Equation 2
$K_{MAPSWAT} = \frac{e}{\sum{\tfrac{e_i}{K_{MAPSWAT_i}}}}$

Assuming the next relationship

$\tfrac{K_{Hydrogeologic}}{K_{MAPSWAT}} = r$

Then 

$\tfrac{K_{Hydrogeologic_i}}{K_{MAPSWAT_i}} = r$

In [81]:
SOIL_map = gpd.read_file("1.PRODUCTOS/SOIL_map.shp")

# 3) Preparar tres copias sin eliminar aún nada
SOIL_map_min  = SOIL_map.copy()
SOIL_map_max  = SOIL_map.copy()
SOIL_map_prom = SOIL_map.copy()

sol_cols = [atri for atri in SOIL_map.columns if atri.startswith('SOL_K')]
hydrogeological_observations = SOIL_map['Min'].notna()

for idx, row in SOIL_map[hydrogeological_observations].iterrows():
    vals = row[sol_cols]
    SOL_K_MAPSWAT = vals.sum()
    r_min  = row['Min'] / SOL_K_MAPSWAT
    r_max  = row['Max'] / SOL_K_MAPSWAT
    r_prom = row['Prom'] / SOL_K_MAPSWAT
    
    SOIL_map_min.loc[idx, vals.index] = row[vals.index] * r_min
    SOIL_map_max.loc[idx, vals.index] = row[vals.index] * r_max
    SOIL_map_prom.loc[idx, vals.index] = row[vals.index] * r_prom

SOIL_map_min = SOIL_map_min.drop(columns = ['NAME','Min','Max','Prom'])
SOIL_map_max = SOIL_map_max.drop(columns = ['NAME','Min','Max','Prom'])
SOIL_map_prom = SOIL_map_prom.drop(columns = ['NAME','Min','Max','Prom'])

SOIL_map_min.to_file("1.PRODUCTOS/SOIL_map_min_v1.shp")
SOIL_map_max.to_file("1.PRODUCTOS/SOIL_map_max_v1.shp")
SOIL_map_prom.to_file("1.PRODUCTOS/SOIL_map_prom_v1.shp")

SOIL_map_min_table  = SOIL_map_min.drop(columns='geometry')
SOIL_map_max_table  = SOIL_map_max.drop(columns='geometry')
SOIL_map_prom_table = SOIL_map_prom.drop(columns='geometry')

SOIL_map_min_table.to_csv("1.PRODUCTOS/SOIL_map_min_v1.csv", index=False)
SOIL_map_max_table.to_csv("1.PRODUCTOS/SOIL_map_max_v1.csv", index=False)
SOIL_map_prom_table.to_csv("1.PRODUCTOS/SOIL_map_prom_v1.csv", index=False)

In [85]:
SOIL_map = gpd.read_file("1.PRODUCTOS/SOIL_map.shp")

# 3) Preparar tres copias sin eliminar aún nada
SOIL_map_min  = SOIL_map.copy()
SOIL_map_max  = SOIL_map.copy()
SOIL_map_prom = SOIL_map.copy()

sol_cols = [atri for atri in SOIL_map.columns if atri.startswith('SOL_K')]
deep_cols = [atri for atri in SOIL_map.columns if atri.startswith('SOL_Z') and not atri.endswith('MX')]
hydrogeological_observations = SOIL_map['Min'].notna()

for idx, row in SOIL_map[hydrogeological_observations].iterrows():
    vals = row[sol_cols]
    deeps = row[deep_cols]
    SOL_Z_MAPSWAT = deeps.sum()
    SOL_K_MAPSWAT = SOL_Z_MAPSWAT/np.sum(deeps.values[deeps.values > 0] / vals.values[deeps.values > 0])
    #print(SOL_K_MAPSWAT)
    #print(vals.values)
    #print(deeps.values[deeps.values > 0] / vals.values[deeps.values > 0])
    r_min  = row['Min'] / SOL_K_MAPSWAT
    r_max  = row['Max'] / SOL_K_MAPSWAT
    r_prom = row['Prom'] / SOL_K_MAPSWAT
    
    SOIL_map_min.loc[idx, vals.index] = row[vals.index] * r_min
    SOIL_map_max.loc[idx, vals.index] = row[vals.index] * r_max
    SOIL_map_prom.loc[idx, vals.index] = row[vals.index] * r_prom
    SOIL_map_min = SOIL_map_min.drop(columns = ['NAME','Min','Max','Prom'])
    
SOIL_map_max = SOIL_map_max.drop(columns = ['NAME','Min','Max','Prom'])
SOIL_map_prom = SOIL_map_prom.drop(columns = ['NAME','Min','Max','Prom'])
#print(SOIL_map_prom)
SOIL_map_min.to_file("1.PRODUCTOS/SOIL_map_min_v2.shp")
SOIL_map_max.to_file("1.PRODUCTOS/SOIL_map_max_v2.shp")
SOIL_map_prom.to_file("1.PRODUCTOS/SOIL_map_prom_v2.shp")

SOIL_map_min_table  = SOIL_map_min.drop(columns='geometry')
SOIL_map_max_table  = SOIL_map_max.drop(columns='geometry')
SOIL_map_prom_table = SOIL_map_prom.drop(columns='geometry')

SOIL_map_min_table.to_csv("1.PRODUCTOS/SOIL_map_min_v2.csv", index=False)
SOIL_map_max_table.to_csv("1.PRODUCTOS/SOIL_map_max_v2.csv", index=False)
SOIL_map_prom_table.to_csv("1.PRODUCTOS/SOIL_map_prom_v2.csv", index=False)